In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import accuracy_score
%matplotlib inline

In [48]:
### define fetching data function 
def get_data():
    aapl = yf.download("AAPL", start="2020-01-01", end="2025-01-01")['Close']
    spy = yf.download("SPY", start="2020-01-01", end="2025-01-01")

    df = pd.DataFrame({'Close': aapl['AAPL']})

    ### Feature 1 - Calculate return as 
    for i in range(1,6):
        df[f'Lag_{i}'] = df['Close'].pct_change(i)

    ### Feature 2 - Rolling std
    #df['Volatility'] = df['Close'].pct_change().rolling(5).std()

    ### Feature 3 - average distance (Price / SMA_20 - 1)
    #df['MA_Dist'] = df['Close'] / df['Close'].rolling(20).mean() - 1

    ### Feature 4 - RSI
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rsi = gain / loss
    df["RSI"] = 100 - (100/(1+rsi))

    ### Feature 5 - SPY Lag 1
    df['SPY_Lag_1'] = spy['Close'].pct_change(1)

    ### 3. Label
    # If close price: tomorrow > today, set 1 (up), else set 0 (down)
    # .shift(-1) is required, moving tomorrow rows to today rows as reference
    df['Target'] = np.where(df['Close'].shift(-1) > df["Close"], 1, 0)

    df.dropna(inplace=True)

    return df

In [49]:
data = get_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [50]:
data

,Close,Lag_1,Lag_2,Lag_3,Lag_4,Lag_5,RSI,SPY_Lag_1,Target
Date,,,,,,,,,
2020-01-22,76.654465,0.003570,-0.003232,0.007804,0.020428,0.016055,70.868368,0.000121,1
2020-01-23,77.023621,0.004816,0.008403,0.001569,0.012657,0.025342,71.902503,0.001147,0
2020-01-24,76.801651,-0.002882,0.001920,0.005497,-0.001318,0.009739,75.401644,-0.008893,0
2020-01-27,74.543251,-0.029406,-0.032203,-0.027542,-0.024071,-0.030685,59.513375,-0.016029,1
2020-01-28,76.652046,0.028290,-0.001948,-0.004824,-0.000032,0.003538,67.412410,0.010479,1
...,...,...,...,...,...,...,...,...,...
2024-12-24,257.037476,0.011478,0.014578,0.033668,0.040919,0.018621,76.180681,0.011115,1
2024-12-26,257.853760,0.003176,0.014690,0.017800,0.036951,0.044225,76.812106,0.000067,0
2024-12-27,254.439240,-0.013242,-0.010108,0.001254,0.004322,0.023220,69.300710,-0.010527,0


In [51]:
###  Split dataset into Train and Test
# Financial data could not be shuffled randomly, it must be divided according to time
# First 80% data as Train set, rest as Test set
split = int(len(data) * 0.8)
train = data.iloc[:split]
test = data.iloc[split:]

In [52]:
features = [col for col in data.columns if col not in ["Close", "Target"]]
X_train, y_train = train[features], train['Target']
X_test, y_test = test[features], test['Target']

In [53]:
### 2. Define grid for parameters searching
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [10, 20, 50],
    'min_samples_leaf': [1, 2, 4]
}

In [54]:
### 3. Searching
# cv = TimeSeriesSplit: Financial data must be validated according temporal order. Random shuffle is not allowed
print("Optimizing HyperParameters, it might take several minutes.")
rf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid,
                           cv=TimeSeriesSplit(n_splits=3), # 3-fold cross validation
                           n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

Optimizing HyperParameters, it might take several minutes.
Fitting 3 folds for each of 81 candidates, totalling 243 fits


,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",RandomForestC...ndom_state=42)
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'max_depth': [None, 10, ...], 'min_samples_leaf': [1, 2, ...], 'min_samples_split': [10, 20, ...], 'n_estimators': [50, 100, ...]}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",None
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",-1
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",TimeSeriesSpl...est_size=None)
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computa

In [55]:
### 4. Display optimal parameters
print("\n========== Optimal Parameters ==========")
print(grid_search.best_params_)


========== Optimal Parameters ==========
{'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 20, 'n_estimators': 100}


In [56]:
### 5. Evaluation
best_model = grid_search.best_estimator_
preds = best_model.predict(X_test)
acc = accuracy_score(y_test, preds)
print(f"Accuracy after optimization: {acc:.2%}")

Accuracy after optimization: 51.41%


### Detect overfitting

In [57]:
### 1. Display optimal parameters
print("\n========== Optimal Parameters ==========")
print(grid_search.best_params_)


========== Optimal Parameters ==========
{'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 20, 'n_estimators': 100}


In [58]:
### 2. Score of Train and Test
train_acc = accuracy_score(y_train, best_model.predict(X_train))
test_acc = accuracy_score(y_test, best_model.predict(X_test))

print(f"\nAccuracy of Train: {train_acc:.2%}")
print(f"\nAccuracy of Test: {test_acc:.2%}")
print(f"\nOverfitting Gap: {train_acc - test_acc:.2%}")


Accuracy of Train: 95.28%

Accuracy of Test: 51.41%

Overfitting Gap: 43.88%


In [59]:
if train_acc - test_acc > 0.15:
    print("Overfitting! Model is not learning!")
else:
    print("Underfitting! There might be some noises in the features.")

Overfitting! Model is not learning!
